In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

#### Load data

In [20]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')
print(train.info(), test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26729 entries, 0 to 26728
Data columns (total 10 columns):
AnimalID          26729 non-null object
Name              19038 non-null object
DateTime          26729 non-null object
OutcomeType       26729 non-null object
OutcomeSubtype    13117 non-null object
AnimalType        26729 non-null object
SexuponOutcome    26728 non-null object
AgeuponOutcome    26711 non-null object
Breed             26729 non-null object
Color             26729 non-null object
dtypes: object(10)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11456 entries, 0 to 11455
Data columns (total 8 columns):
ID                11456 non-null int64
Name              8231 non-null object
DateTime          11456 non-null object
AnimalType        11456 non-null object
SexuponOutcome    11456 non-null object
AgeuponOutcome    11450 non-null object
Breed             11456 non-null object
Color             11456 non-null object
dtypes: int64(1), object

In our training data, we are missing data for:
    - Name -- no problem
    - OutcomeSubtype - we will drop this field
    - SexuponOutcome -- only 1
    - AgeuponOutcome -- only a handful


## Clean Data

In [21]:
#copy data
df_train = train.copy()
df_test = test.copy()

### Clean DateTime

In [22]:
#format datetimes
df_train.DateTime = pd.to_datetime(df_train.DateTime)
df_test.DateTime = pd.to_datetime(df_test.DateTime)

In [23]:
#extract year
df_train['Year'] = [x.year for x in df_train.DateTime]
df_test['Year'] = [x.year for x in df_test.DateTime]

#extract month
df_train['Month'] = [x.month for x in df_train.DateTime]
df_test['Month'] = [x.month for x in df_test.DateTime]

### Named?

In [24]:
#check if named
df_train['Named'] = df_train.Name.notnull().astype(int)
df_test['Named'] = df_train.Name.notnull().astype(int)

### Clean 'Sex'

There is only one missing value for 'SexuponOutcome'.  Upon further investigation, that animal's name was Diego. As such, we will assume it is male.

In [25]:
df_train[df_train.SexuponOutcome.isnull()]

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Year,Month,Named
3174,A667395,Diego,2013-11-27 16:11:00,Return_to_owner,NaN,Dog,NaN,7 years,Dachshund,Brown Merle,2013,11,1


In [26]:
#fill missing SexuponOutcome
df_train = df_train.set_value(3174,'SexuponOutcome','Male') #name is Diego, so we assume male

In [27]:
def clean_sex(data):
    #create features by splitting SexuponOutcome 
    data['Sex'] = data['SexuponOutcome'].str.split().str[-1]
    data['Fixed Status'] = data['SexuponOutcome'].str.split().str[0]
    
    data.drop('SexuponOutcome', axis=1, inplace=True)
    
    #Assume no details means no surgery done
    data['Fixed Status'] = data['Fixed Status'].str.replace('Male','Intact')
    data['Fixed Status'] = data['Fixed Status'].str.replace('Unknown','Intact')
    
    return data

In [28]:
df_train = clean_sex(df_train)
df_test = clean_sex(df_test)

### Clean 'Age'

In [29]:
#fill in missing age field values
df_train.AgeuponOutcome = df_train.AgeuponOutcome.fillna('1000 years') #placeholder -- 1000 is not a possible age
df_test.AgeuponOutcome = df_test.AgeuponOutcome.fillna('1000 years')   #placeholder -- 1000 is not a possible age

In [30]:
def clean_age(row):
    #standardizes age units to day
    if 'year' in str(row['AgeuponOutcome'][1]):
        return float(row['AgeuponOutcome'][0]) * 365
    if 'month' in str(row['AgeuponOutcome'][1]):
        return float(row['AgeuponOutcome'][0]) * 30
    if 'week' in str(row['AgeuponOutcome'][1]):
        return float(row['AgeuponOutcome'][0]) * 7
    if 'day' in str(row['AgeuponOutcome'][1]):
        return float(row['AgeuponOutcome'][0])
    if row['AgeuponOutcome'].isnull():
        return row['AgeuponOutcome']
    else:
        return np.nan

In [31]:
#prepare age
df_train['AgeuponOutcome'] = df_train['AgeuponOutcome'].str.split()
df_test['AgeuponOutcome'] = df_test['AgeuponOutcome'].str.split()

#standardize age
df_train['Age'] = df_train.apply(clean_age, axis=1) 
df_test['Age'] = df_test.apply(clean_age, axis=1)

#convert age to year
df_train['Age'] = df_train['Age'] / 365
df_test['Age'] = df_test['Age'] / 365

The test set has missing age values, which we will fix by imputing the median age.

In [32]:
#test set has missing age values -- impute with median age

#replace placeholders
ages = []
for age in df_test.Age:
    if age > 40:
        ages.append(df_train.Age.median())
    else:
        ages.append(age)

df_test['Age'] = pd.Series(ages)

### Clean Color

In [33]:
#get unique colors
unique_colors_train = sorted(list(pd.Series([item for sublist in df_train.Color.str.split('/') for item in sublist]).unique()))
unique_colors_train = [c.lower().replace(' ','_') for c in unique_colors_train]

unique_colors_test = sorted(list(pd.Series([item for sublist in df_test.Color.str.split('/') for item in sublist]).unique()))
unique_colors_test = [c.lower().replace(' ','_') for c in unique_colors_test]

unique_colors = unique_colors_train + unique_colors_test
unique_colors = pd.Series(unique_colors).unique()

#create color features
for color in unique_colors:
    df_train[color] = 0
    df_test[color] = 0

In [34]:
#clean colors
df_train['Color'] = [color.replace(' ','_').lower().split('/') for color in df_train.Color]
df_test['Color'] = [color.replace(' ','_').lower().split('/') for color in df_test.Color]

In [35]:
#fill features values
for color in unique_colors:
    
    try:
        df_train[color] = df_train['Color'].apply(lambda x: color in x ).astype(int)
        df_test[color] = df_test['Color'].apply(lambda x: color in x ).astype(int)
    except:
        print('Error',color)

### Clean Breed

In [36]:
def mix_test(row):
    if 'Mix' in row['Breed']:
        return 1
    else:
        return 0

In [37]:
#check if Breed included 'Mix'
df_train['Mix'] = df_train.apply(mix_test,axis=1)
df_test['Mix'] = df_test.apply(mix_test,axis=1)

In [38]:
#clean Breed field
df_train['Breed'] = df_train.Breed.str.replace(' ','_').str.lower().str.strip('_mix').str.split('/')
df_test['Breed'] = df_test.Breed.str.replace(' ','_').str.lower().str.strip('_mix').str.split('/')

In [40]:
#retrieve unique breeds
unique_breeds_train = sorted(list(pd.Series([item for sublist in df_train.Breed for item in sublist]).unique()))
unique_breeds_test = sorted(list(pd.Series([item for sublist in df_test.Breed for item in sublist]).unique()))
unique_breeds = unique_breeds_train + unique_breeds_test
unique_breeds = pd.Series(unique_breeds).unique()

In [41]:
#create empty fields for breeds
for breed in unique_breeds:
    df_train[breed] = 0
    df_test[breed] = 0

#check for breed
for breed in unique_breeds:
    df_train[breed] = df_train['Breed'].apply(lambda x: breed in x ).astype(int)
    df_test[breed] = df_test['Breed'].apply(lambda x: breed in x ).astype(int)

### Feature Match Sanity Check

In [42]:
#training fields not in testing set
mask = [x not in df_test.columns for x in df_train.columns]
list(pd.Series(df_train.columns)[mask].values)

['AnimalID', 'OutcomeType', 'OutcomeSubtype']

In [43]:
#training fields not in testing set
mask = [x not in df_train.columns for x in df_test.columns]
list(pd.Series(df_test.columns)[mask].values)

['ID']

### Convert Categorical Features to Binary Fields

In [44]:
animal_train = pd.get_dummies(df_train['AnimalType'])
animal_test = pd.get_dummies(df_test['AnimalType'])

sex_train = pd.get_dummies(df_train['Sex'])
sex_test = pd.get_dummies(df_test['Sex'])

fixed_status_train = pd.get_dummies(df_train['Fixed Status'])
fixed_status_test = pd.get_dummies(df_test['Fixed Status'])

In [45]:
df_train = pd.concat([df_train, animal_train, sex_train, fixed_status_train], axis=1)
df_train = df_train.drop(['AnimalType','Sex','Fixed Status','Breed','Color','Name','DateTime','AgeuponOutcome'], axis=1)

df_test = pd.concat([df_test, animal_test, sex_test, fixed_status_test], axis=1)
df_test = df_test.drop(['AnimalType','Sex','Fixed Status','Breed','Color','Name','DateTime','AgeuponOutcome'], axis=1)

## Cleaned Dataset

In [47]:
df_train.to_csv('clean_train_v2.csv')
df_test.to_csv('clean_test_v2.csv')